<a href="https://colab.research.google.com/github/vivekgautamgv/Algorithmic-Trading-with-Python-Cookbook/blob/main/renko_chart_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ccxt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.0/131.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.3/289.3 kB 24.8 MB/s eta 0:00:00
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.15
    Uninstalling aiohttp-3.11.15:
      Successfully uninstalled aiohttp-3.11.15


In [ ]:
!pip install pandas_ta

In [ ]:
! pip install pandas-ta

In [1]:
!pip install yfinance pandas matplotlib renko


In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime, timedelta
import plotly.graph_objects as go

class RenkoStrategy:
    def __init__(self, symbol, brick_size, start_date, end_date):
        self.symbol = symbol
        self.brick_size = brick_size
        self.start_date = start_date
        self.end_date = end_date
        self.data = None
        self.renko_data = None
        self.signals = None
        self.trades = []
        self.performance = {}

    def fetch_data(self):
        """Fetch OHLC data from Yahoo Finance"""
        data = yf.download(self.symbol, start=self.start_date, end=self.end_date, interval="5m")
        self.data = data.copy()
        return data

    def construct_renko(self):
        """Construct Renko chart from OHLC data"""
        # Initialize renko data
        renko = pd.DataFrame(columns=['Date', 'Open', 'High', 'Low', 'Close', 'Direction'])

        if self.data is None:
            print("No data to construct Renko chart. Please fetch data first.")
            return None

        close_prices = self.data['Close'].values

        # Initialize first brick
        open_price = close_prices[0]
        direction = 0
        brick_count = 0

        # Create renko bricks
        for i, close_price in enumerate(close_prices):
            date_time = self.data.index[i]

            # First brick
            if brick_count == 0:
                # Determine direction of first brick
                if close_price >= open_price + self.brick_size:
                    # Bullish brick
                    direction = 1
                    brick_high = open_price + self.brick_size
                    brick_low = open_price
                    renko = pd.concat([renko, pd.DataFrame({'Date': [date_time],
                                                    'Open': [open_price],
                                                    'High': [brick_high],
                                                    'Low': [brick_low],
                                                    'Close': [brick_high],
                                                    'Direction': [direction]})], ignore_index=True)
                    open_price = brick_high
                    brick_count += 1
                elif close_price <= open_price - self.brick_size:
                    # Bearish brick
                    direction = -1
                    brick_high = open_price
                    brick_low = open_price - self.brick_size
                    renko = pd.concat([renko, pd.DataFrame({'Date': [date_time],
                                                    'Open': [open_price],
                                                    'High': [brick_high],
                                                    'Low': [brick_low],
                                                    'Close': [brick_low],
                                                    'Direction': [direction]})], ignore_index=True)
                    open_price = brick_low
                    brick_count += 1

            # Subsequent bricks
            else:
                # Check if we can draw a new brick
                if direction == 1:
                    # Check for continuation or reversal
                    if close_price >= open_price + self.brick_size:
                        # Continuation bullish brick
                        brick_high = open_price + self.brick_size
                        brick_low = open_price
                        renko = pd.concat([renko, pd.DataFrame({'Date': [date_time],
                                                        'Open': [open_price],
                                                        'High': [brick_high],
                                                        'Low': [brick_low],
                                                        'Close': [brick_high],
                                                        'Direction': [direction]})], ignore_index=True)
                        open_price = brick_high
                        brick_count += 1
                    elif close_price <= open_price - 2*self.brick_size:
                        # Reversal - needs 2 brick size movement for reversal
                        direction = -1
                        # Add reversal brick
                        brick_high = open_price
                        brick_low = open_price - self.brick_size
                        renko = pd.concat([renko, pd.DataFrame({'Date': [date_time],
                                                        'Open': [open_price],
                                                        'High': [brick_high],
                                                        'Low': [brick_low],
                                                        'Close': [brick_low],
                                                        'Direction': [direction]})], ignore_index=True)
                        open_price = brick_low
                        brick_count += 1

                        # Check if we need to add another brick
                        if close_price <= open_price - self.brick_size:
                            brick_high = open_price
                            brick_low = open_price - self.brick_size
                            renko = pd.concat([renko, pd.DataFrame({'Date': [date_time],
                                                            'Open': [open_price],
                                                            'High': [brick_high],
                                                            'Low': [brick_low],
                                                            'Close': [brick_low],
                                                            'Direction': [direction]})], ignore_index=True)
                            open_price = brick_low
                            brick_count += 1

                elif direction == -1:
                    # Check for continuation or reversal
                    if close_price <= open_price - self.brick_size:
                        # Continuation bearish brick
                        brick_high = open_price
                        brick_low = open_price - self.brick_size
                        renko = pd.concat([renko, pd.DataFrame({'Date': [date_time],
                                                        'Open': [open_price],
                                                        'High': [brick_high],
                                                        'Low': [brick_low],
                                                        'Close': [brick_low],
                                                        'Direction': [direction]})], ignore_index=True)
                        open_price = brick_low
                        brick_count += 1
                    elif close_price >= open_price + 2*self.brick_size:
                        # Reversal - needs 2 brick size movement for reversal
                        direction = 1
                        # Add reversal brick
                        brick_high = open_price + self.brick_size
                        brick_low = open_price
                        renko = pd.concat([renko, pd.DataFrame({'Date': [date_time],
                                                        'Open': [open_price],
                                                        'High': [brick_high],
                                                        'Low': [brick_low],
                                                        'Close': [brick_high],
                                                        'Direction': [direction]})], ignore_index=True)
                        open_price = brick_high
                        brick_count += 1

                        # Check if we need to add another brick
                        if close_price >= open_price + self.brick_size:
                            brick_high = open_price + self.brick_size
                            brick_low = open_price
                            renko = pd.concat([renko, pd.DataFrame({'Date': [date_time],
                                                            'Open': [open_price],
                                                            'High': [brick_high],
                                                            'Low': [brick_low],
                                                            'Close': [brick_high],
                                                            'Direction': [direction]})], ignore_index=True)
                            open_price = brick_high
                            brick_count += 1

        self.renko_data = renko
        return renko

    def generate_signals(self):
        """Generate trading signals based on 3 consecutive bricks strategy"""
        if self.renko_data is None or len(self.renko_data) < 3:
            print("Not enough Renko data to generate signals")
            return None

        # Initialize signal column
        self.renko_data['Signal'] = 0

        # Check for 3 consecutive bricks
        for i in range(3, len(self.renko_data)):
            # Check for 3 consecutive up bricks (value of 1)
            if (self.renko_data.iloc[i-3]['Direction'] == 1 and
                self.renko_data.iloc[i-2]['Direction'] == 1 and
                self.renko_data.iloc[i-1]['Direction'] == 1):
                # Long signal for the 4th brick
                self.renko_data.loc[self.renko_data.index[i], 'Signal'] = 1

            # Check for 3 consecutive down bricks (value of -1)
            elif (self.renko_data.iloc[i-3]['Direction'] == -1 and
                  self.renko_data.iloc[i-2]['Direction'] == -1 and
                  self.renko_data.iloc[i-1]['Direction'] == -1):
                # Short signal for the 4th brick
                self.renko_data.loc[self.renko_data.index[i], 'Signal'] = -1

        self.signals = self.renko_data[self.renko_data['Signal'] != 0].copy()
        return self.signals

    def backtest(self):
        """Backtest the strategy and calculate performance metrics"""
        if self.signals is None or len(self.signals) == 0:
            print("No signals generated. Cannot perform backtest.")
            return

        # Initialize trade tracking
        self.trades = []
        initial_capital = 100000  # Starting with $100,000
        current_capital = initial_capital

        for i, row in self.signals.iterrows():
            entry_price = row['Open']
            exit_price = row['Close']

            if row['Signal'] == 1:  # Long trade
                profit_loss = (exit_price - entry_price) / entry_price * current_capital
                trade_type = 'LONG'
            else:  # Short trade
                profit_loss = (entry_price - exit_price) / entry_price * current_capital
                trade_type = 'SHORT'

            # Record the trade
            trade = {
                'Date': row['Date'],
                'Type': trade_type,
                'Entry': entry_price,
                'Exit': exit_price,
                'PnL': profit_loss,
                'PnL%': (profit_loss / current_capital) * 100
            }
            self.trades.append(trade)

            # Update capital
            current_capital += profit_loss

        # Convert trades to DataFrame
        self.trades_df = pd.DataFrame(self.trades)

        # Calculate performance metrics
        total_trades = len(self.trades)
        winning_trades = len(self.trades_df[self.trades_df['PnL'] > 0])
        losing_trades = len(self.trades_df[self.trades_df['PnL'] <= 0])
        win_rate = (winning_trades / total_trades) * 100 if total_trades > 0 else 0

        total_profit = self.trades_df['PnL'].sum()
        avg_profit_per_trade = self.trades_df['PnL'].mean()

        long_trades = len(self.trades_df[self.trades_df['Type'] == 'LONG'])
        short_trades = len(self.trades_df[self.trades_df['Type'] == 'SHORT'])

        long_profits = self.trades_df[self.trades_df['Type'] == 'LONG']['PnL'].sum()
        short_profits = self.trades_df[self.trades_df['Type'] == 'SHORT']['PnL'].sum()

        max_drawdown = self.calculate_max_drawdown(current_capital, self.trades_df)

        # Store performance metrics
        self.performance = {
            'Initial Capital': initial_capital,
            'Final Capital': current_capital,
            'Total Return': (current_capital - initial_capital) / initial_capital * 100,
            'Total Trades': total_trades,
            'Winning Trades': winning_trades,
            'Losing Trades': losing_trades,
            'Win Rate': win_rate,
            'Total Profit/Loss': total_profit,
            'Average Profit/Loss Per Trade': avg_profit_per_trade,
            'Long Trades': long_trades,
            'Short Trades': short_trades,
            'Long Trades P/L': long_profits,
            'Short Trades P/L': short_profits,
            'Max Drawdown %': max_drawdown
        }

        return self.performance

    def calculate_max_drawdown(self, final_capital, trades_df):
        """Calculate maximum drawdown percentage"""
        if trades_df.empty:
            return 0

        # Calculate cumulative P&L
        trades_df['Cumulative_PnL'] = trades_df['PnL'].cumsum()

        # Initialize account value with initial capital
        account_values = [100000]  # Starting with initial capital

        # Build account value history
        for pnl in trades_df['PnL']:
            account_values.append(account_values[-1] + pnl)

        # Calculate drawdown
        peak = account_values[0]
        max_dd = 0

        for value in account_values:
            if value > peak:
                peak = value
            dd = (peak - value) / peak * 100
            if dd > max_dd:
                max_dd = dd

        return max_dd

    def plot_renko_with_signals(self):
        """Plot Renko chart with signals"""
        if self.renko_data is None:
            print("No Renko data to plot")
            return

        # Create figure
        fig = go.Figure()

        # Add Renko chart
        for i, row in self.renko_data.iterrows():
            if row['Direction'] == 1:
                color = 'green'
            else:
                color = 'red'

            # Draw brick
            fig.add_shape(
                type="rect",
                x0=i-0.4, y0=row['Low'],
                x1=i+0.4, y1=row['High'],
                fillcolor=color,
                line=dict(color=color),
                opacity=0.7
            )

        # Add signals
        for i, row in self.signals.iterrows():
            idx = self.renko_data.index.get_loc(i)

            if row['Signal'] == 1:  # Long signal
                fig.add_trace(go.Scatter(
                    x=[idx],
                    y=[row['Low'] - self.brick_size/2],
                    mode='markers',
                    marker=dict(size=12, color='blue', symbol='triangle-up'),
                    name='Long Signal'
                ))
            else:  # Short signal
                fig.add_trace(go.Scatter(
                    x=[idx],
                    y=[row['High'] + self.brick_size/2],
                    mode='markers',
                    marker=dict(size=12, color='purple', symbol='triangle-down'),
                    name='Short Signal'
                ))

        # Update layout
        fig.update_layout(
            title=f"Renko Chart for {self.symbol} with Trading Signals",
            xaxis_title="Brick Number",
            yaxis_title="Price",
            showlegend=True
        )

        fig.update_xaxes(range=[-1, len(self.renko_data) + 1])

        return fig

    def plot_equity_curve(self):
        """Plot equity curve"""
        if not hasattr(self, 'trades_df') or self.trades_df.empty:
            print("No trades to plot")
            return

        # Calculate cumulative P&L
        self.trades_df['Cumulative_PnL'] = self.trades_df['PnL'].cumsum()

        # Create figure
        fig = go.Figure()

        # Initial capital
        initial_capital = 100000

        # Create equity values
        equity_values = [initial_capital]
        for pnl in self.trades_df['PnL']:
            equity_values.append(equity_values[-1] + pnl)

        # Add equity curve
        fig.add_trace(go.Scatter(
            x=list(range(len(equity_values))),
            y=equity_values,
            mode='lines',
            name='Equity Curve'
        ))

        # Update layout
        fig.update_layout(
            title=f"Equity Curve for {self.symbol}",
            xaxis_title="Trade Number",
            yaxis_title="Equity ($)",
            showlegend=True
        )

        return fig

    def run_full_backtest(self):
        """Run the full backtest process"""
        print(f"Fetching data for {self.symbol}...")
        self.fetch_data()

        print("Constructing Renko chart...")
        self.construct_renko()

        print("Generating trading signals...")
        self.generate_signals()

        print("Running backtest...")
        performance = self.backtest()

        print("\n=== BACKTEST RESULTS ===")
        for key, value in performance.items():
            if isinstance(value, float):
                print(f"{key}: {value:.2f}")
            else:
                print(f"{key}: {value}")

        return performance

# Example usage
if __name__ == "__main__":
    # Define backtest parameters
    symbol = "ADANIPOWER.NS"  # Example ticker
    brick_size = 1.90 # Size of each Renko brick in dollars
    start_date = datetime.now() - timedelta(days=30)  # Last 30 days
    end_date = datetime.now()

    # Create and run the strategy
    strategy = RenkoStrategy(symbol, brick_size, start_date, end_date)
    performance = strategy.run_full_backtest()

    # Plot Renko chart with signals
    renko_fig = strategy.plot_renko_with_signals()
    renko_fig.show()

    # Plot equity curve
    equity_fig = strategy.plot_equity_curve()
    equity_fig.show()

    # Print trade details
    if hasattr(strategy, 'trades_df') and not strategy.trades_df.empty:
        print("\n=== TRADE DETAILS ===")
        print(strategy.trades_df)

[*********************100%***********************]  1 of 1 completed

Fetching data for ADANIPOWER.NS...
Constructing Renko chart...



<ipython-input-19-1cbb0750d6cf>:67: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



Generating trading signals...
Running backtest...

=== BACKTEST RESULTS ===
Initial Capital: 100000
Final Capital: [144009.19234767]
Total Return: [44.00919235]
Total Trades: 158
Winning Trades: 129
Losing Trades: 29
Win Rate: 81.65
Total Profit/Loss: [44009.19234767]
Average Profit/Loss Per Trade: [278.53919207]
Long Trades: 86
Short Trades: 72
Long Trades P/L: [26177.1732758]
Short Trades P/L: [17832.01907187]
Max Drawdown %: [1.10435026]



=== TRADE DETAILS ===
                         Date   Type                Entry  \
0   2025-03-20 04:35:00+00:00  SHORT  [519.9500244140626]   
1   2025-03-20 04:40:00+00:00  SHORT  [518.0500244140626]   
2   2025-03-20 04:55:00+00:00  SHORT  [516.1500244140626]   
3   2025-03-21 04:05:00+00:00   LONG  [521.8500244140625]   
4   2025-03-21 04:55:00+00:00   LONG  [523.7500244140625]   
..                        ...    ...                  ...   
153 2025-04-17 05:00:00+00:00   LONG  [548.4500244140622]   
154 2025-04-17 05:05:00+00:00   LONG  [550.3500244140622]   
155 2025-04-17 05:10:00+00:00   LONG  [552.2500244140622]   
156 2025-04-17 05:40:00+00:00   LONG  [554.1500244140622]   
157 2025-04-17 08:05:00+00:00  SHORT  [548.4500244140622]   

                    Exit                    PnL                   PnL%  \
0    [518.0500244140626]   [365.41973474106635]  [0.36541973474106637]   
1    [516.1500244140626]    [368.1001611990838]  [0.36675994797007505]   
2    [518.050024414062